# Building Agents 
 
> Note: Optionally, see [these slides](https://docs.google.com/presentation/d/13c0L1CQWAL7fuCXakOqjkvoodfynPJI4Hw_4H76okVU/edit?usp=sharing) and [langgraph_101.ipynb](langgraph_101.ipynb) for context before diving into this notebook!

We're going to build an email assistant from scratch, starting here with 1) the agent architecture (using [LangGraph JS](https://langchain-ai.github.io/langgraphjs/)) and following with 2) testing (using [LangSmith](https://docs.smith.langchain.com/)), 3) human-in-the-loop, and 4) memory. This diagram show how these pieces will fit together:

![overview-img](img/overview.png)

### Tool Definition

Let's start by defining some simple tools that an email assistant will use with the `@tool` decorator:

In [1]:
import { z } from "zod";
// import
import { tool } from "@langchain/core/tools";

// Define schema for email writing
const writeEmailSchema = z.object({
  to: z.string().describe("Email address of the recipient"),
  subject: z.string().describe("Subject line for the email"),
  content: z.string().describe("Main body text of the email")
});

// Write email tool
const writeEmail = tool(async ( to, subject, content ) => {
  // Placeholder response - in real app would send email
  return `Email sent to ${to} with subject '${subject}' and content: ${content}`;
}, {
  name: "write_email",
  description: "Write and send an email.",
  schema: writeEmailSchema,
})



// Define schema for scheduling meeting
const scheduleMeetingSchema = z.object({
  attendees: z.array(z.string()).describe("List of attendees' emails"),
  subject: z.string().describe("Meeting title or subject"),
  duration_minutes: z.number().describe("Duration of meeting in minutes"),
  preferred_day: z.string().describe("Preferred date for the meeting"),
  start_time: z.number().describe("Start time of the meeting")
});

// Schedule meeting tool
export const scheduleMeeting = new DynamicStructuredTool({
  name: "schedule_meeting",
  description: "Schedule a calendar meeting.",
  schema: scheduleMeetingSchema,
  func: async (attendees, subject, duration_minutes, preferred_day, start_time )=> {
    // Placeholder response - in real app would check calendar and schedule
    const dateStr = new Date(preferred_day).toLocaleDateString('en-US', { weekday: 'long', month: 'long', day: 'numeric', year: 'numeric' });
    return `Meeting '${subject}' scheduled on ${dateStr} at ${start_time} for ${duration_minutes} minutes with ${attendees.length} attendees`;
  }
});

// Define schema for checking calendar
const checkCalendarSchema = z.object({
  day: z.string().describe("Day to check calendar availability")
});

// Check calendar availability tool
export const checkCalendarAvailability = new DynamicStructuredTool({
  name: "check_calendar_availability",
  description: "Check calendar availability for a given day.",
  schema: checkCalendarSchema,
  func: async (day) => {
    // Placeholder response - in real app would check actual calendar
    return `Available times on ${day}: 9:00 AM, 2:00 PM, 4:00 PM`;
  }
});

// Define schema for Done tool
const doneSchema = z.object({
  done: z.boolean().describe("Task completion status")
});
// TODO FIX TOOl
// Done tool
export const Done = new DynamicStructuredTool({
  name: "Done",
  description: "E-mail has been sent.",
  schema: doneSchema,
  func: async () => {
    return "Task completed successfully";
  }
});


13:25 - No overload matches this call.
13:25 - Overload 1 of 3, '(func: RunnableFunc<string, any, ToolRunnableConfig>, fields: ToolWrapperParams<ZodString>): DynamicTool', gave the following error.
13:25 - Argument of type '(to: any, subject: any, content: any) => Promise<string>' is not assignable to parameter of type 'RunnableFunc<string, any, ToolRunnableConfig>'.
13:25 - Overload 2 of 3, '(func: RunnableFunc<any, any, ToolRunnableConfig>, fields: ToolWrapperParams<ZodObject<{ to: ZodString; subject: ZodString; content: ZodString; }, "strip", ZodTypeAny, { ...; }, { ...; }>>): DynamicStructuredTool<...>', gave the following error.
13:25 - Argument of type '(to: any, subject: any, content: any) => Promise<string>' is not assignable to parameter of type 'RunnableFunc<any, any, ToolRunnableConfig>'.
13:25 - Overload 3 of 3, '(func: RunnableFunc<{ to?: string; subject?: string; content?: string; }, any, ToolRunnableConfig>, fields: ToolWrapperParams<ZodObject<{ to: ZodString; subject: Z

### Augmenting the LLM with Tools

Now we connect these tools to a LLM. We'll use LangChain's [`initChatModel`](https://js.langchain.com/docs/how_to/chat_models_universal_init/) interface, which allows us to initialize many different chat models.

We [enforce tool use](https://js.langchain.com/docs/how_to/tool_choice/) by setting `tool_choice: "required"` (or the equivalent parameter for the specific model integration, like in `bindTools`).

In [ ]:

import { initChatModel } from "langchain/chat_models/universal";

// Initialize the LLM
const llm = initChatModel(
  "openai:gpt-4o",
  {temperature: 0.0}
);

// Bind tools to the LLM
const llmWithTools = llm.bindTools([writeEmail]);

// Email input
const emailInput = {
  author: "System Admin <sysadmin@company.com>",
  to: "Development Team <dev@company.com>",
  subject: "Scheduled maintenance - database downtime",
  email_thread: "Hi team,\n\nThis is a reminder that we'll be performing scheduled maintenance on the production database tonight from 2AM to 4AM EST. During this time, all database services will be unavailable.\n\nPlease plan your work accordingly and ensure no critical deployments are scheduled during this window.\n\nThanks,\nSystem Admin Team"
};

// Run the agent
const messages = [{ role: "user", content: "Call the 'write_email' tool in order to respond to this email: " + JSON.stringify(emailInput, null, 2) }];
const output = await llmWithTools.invoke(messages);


In [ ]:
console.log(output.tool_calls);

In [8]:
const args = output.tool_calls?.[0]?.args;


In [ ]:

const writeEmailResult = await writeEmail.invoke(args);
console.log(writeEmailResult);

## Building our email assistant

We'll combine a [router and agent](https://langchain-ai.github.io/langgraphjs/tutorials/workflows/) to build our email assistant.

![agent_workflow_img](img/email_workflow.png)

### Router

The routing step handles the triage decision. 

The triage router only focuses on the triage decision, while the agent focuses *only* on the response. 

### State

When building an agent, it's important to consider the information that you want to track over time. We'll use LangGraph's pre-built [`MessagesState` type (or a similar state management concept described in the [LangGraph JS documentation](https://langchain-ai.github.io/langgraphjs/concepts/low_level/)) which is often a dictionary with a `messages` key that appends messages returned by nodes [as its update logic (see reducers or state updaters in the JS docs)](https://langchain-ai.github.io/langgraphjs/concepts/low_level/). However, LangGraph gives you flexibility to track other information. We'll define a custom `State` object (typically using Zod for schema definition in TypeScript) that might extend a base message state and adds a `classification_decision` key:

In [ ]:
import { z } from "zod";

// to do replace with Proper State for Messages 
// Create StateSchema using Zod
const StateSchema = 
z.object({
  messages: z.array(z.any()),
  email_input: z.record(z.any()),
  classification_decision: z.enum(["ignore", "respond", "notify"])
});




#### Triage node

We define a TypeScript function with our triage routing logic.

For this, we use [structured outputs](https://js.langchain.com/docs/concepts/structured_outputs/) with a Zod schema (instead of Pydantic models used in Python). Zod schemas are particularly useful for defining structured output schemas in TypeScript because they offer type hints and validation. The descriptions in the Zod schema are important because they get passed as part of the JSON schema to the LLM to inform the output coercion (often using the `.withStructuredOutput()` method).

In [ ]:

import { END } from "@langchain/langgraph";
import { ChatOpenAI } from "@langchain/openai";
import { StructuredOutputParser } from "langchain/output_parsers";
import { z } from "zod";

// Import utilities and prompts
import { parseEmail, formatEmailMarkdown } from "../../lib/utils";
import { 
  triageSystemPrompt, 
  triageUserPrompt, 
  defaultTriageInstructions, 
  defaultBackground
} from "../../lib/prompts";

// Define the router schema using Zod
const RouterSchema = z.object({
  reasoning: z.string().describe("Step-by-step reasoning behind the classification."),
  classification: z.enum(["ignore", "respond", "notify"]).describe(
    "The classification of an email: 'ignore' for irrelevant emails, " +
    "'notify' for important information that doesn't need a response, " +
    "'respond' for emails that need a reply"
  )
});

// Initialize the LLM for use with router / structured output
const llm = new ChatOpenAI({
  modelName: "gpt-4-turbo",
  temperature: 0
});

// Create structured output parser
const outputParser = StructuredOutputParser.fromZodSchema(RouterSchema);
const llmRouter = llm.withStructuredOutput(RouterSchema);

// Triage router function
async function triageRouter() {
  // Parse email components
  const { author, to, subject, email_thread } = parseEmail(state.email_input);
  
  // Format prompts
  const systemPrompt = triageSystemPrompt
    .replace("{background}", defaultBackground)
    .replace("{triage_instructions}", defaultTriageInstructions);

  const userPrompt = triageUserPrompt
    .replace("{author}", author)
    .replace("{to}", to)
    .replace("{subject}", subject)
    .replace("{email_thread}", email_thread);

  // Invoke the LLM with structured output
  const result = await llmRouter.invoke([
    { role: "system", content: systemPrompt },
    { role: "user", content: userPrompt }
  ]);

  if (result.classification === "respond") {
    console.log("📧 Classification: RESPOND - This email requires a response");
    return {
      goto: "response_agent",
      update: {
        messages: [
          {
            role: "user",
            content: `Respond to the email: \n\n${formatEmailMarkdown(subject, author, to, email_thread)}`
          }
        ],
        classification_decision: result.classification
      }
    };
  } else if (result.classification === "ignore") {
    console.log("🚫 Classification: IGNORE - This email can be safely ignored");
    return {
      goto: END,
      update: {
        classification_decision: result.classification
      }
    };
  } else if (result.classification === "notify") {
    console.log("🔔 Classification: NOTIFY - This email contains important information");
    // For now, we go to END. But we will add to this later!
    return {
      goto: END,
      update: {
        classification_decision: result.classification
      }
    };
  } else {
    throw new Error(`Invalid classification: ${result.classification}`);
  }
}


We use concepts similar to [Commands (how to update state and jump to nodes)](https://langchain-ai.github.io/langgraphjs/how-tos/command/) in LangGraph to both update the state and select the next node to visit. This is a useful alternative to edges.

### Agent

Now, let's build the agent.

#### LLM node

Here, we define the LLM decision-making node. This node takes in the current state, calls the LLM, and updates `messages` with the LLM output. 

In [12]:
import { AGENT_TOOLS_PROMPT } from "../../lib/tools/default/prompt_templates";
import { 
  agentSystemPrompt, 
  defaultResponsePreferences, 
  defaultCalPreferences 
} from "../../lib/prompts";

In [ ]:
console.log(AGENT_TOOLS_PROMPT);

In [ ]:
console.log(agentSystemPrompt);

In [15]:

// Collect all tools
const tools = [writeEmail, scheduleMeeting, checkCalendarAvailability, Done];
const toolsByName = new Map(tools.map(tool => [tool.name, tool]));

// Initialize the LLM, enforcing tool use
const llm = new ChatOpenAI({
  modelName: "gpt-4-turbo",
  temperature: 0
});

const llmWithTools = llm.bindTools(tools, { toolChoice: "required" });

// LLM call function
async function llmCall() {
  // Format the system prompt with all required information
  const formattedSystemPrompt = agentSystemPrompt
    .replace("{tools_prompt}", AGENT_TOOLS_PROMPT)
    .replace("{background}", defaultBackground)
    .replace("{response_preferences}", defaultResponsePreferences)
    .replace("{cal_preferences}", defaultCalPreferences)
    .replace("{triage_instructions}", defaultTriageInstructions);

  // Invoke the LLM with system prompt and current messages
  const result = await llmWithTools.invoke([
    { role: "system", content: formattedSystemPrompt },
    ...state.messages
  ]);
  
  return {
    messages: [result]
  };
}


#### Tool handler node

After the LLM makes a decision, we need to execute the chosen tool. The `toolHandler` node executes the tool. We can see that nodes can update the graph state to capture any important state changes, such as the classification decision.

In [16]:
// Tool handler function
async function toolHandler() {
  // List for tool messages
  const result = [];
  
  // Get the last message
  const lastMessage = state.messages[state.messages.length - 1];
  
  // Iterate through tool calls
  for (const toolCall of lastMessage.tool_calls || []) {
    // Get the tool
    const tool = toolsByName.get(toolCall.name);
    
    if (!tool) {
      throw new Error(`Tool ${toolCall.name} not found`);
    }
    
    // Run it
    const observation = await tool.invoke(toolCall.args);
    
    // Create a tool message
    result.push({
      role: "tool",
      content: observation,
      tool_call_id: toolCall.id
    });
  }
  
  // Add it to our messages
  return { messages: result };
}

#### Conditional Routing

Our agent needs to decide when to continue using tools and when to stop. This conditional routing function directs the agent to either continue or terminate.

In [17]:
// Function to determine if we should continue or end
function shouldContinue() {
  // Get the last message
  const messages = state.messages;
  const lastMessage = messages[messages.length - 1];
  
  // If the last message is a tool call, check if it's a Done tool call
  if (lastMessage.tool_calls && lastMessage.tool_calls.length > 0) {
    for (const toolCall of lastMessage.tool_calls) {
      if (toolCall.name === "Done") {
        return END;
      }
    }
    return "tool_handler";
  }
  
  // Default case - shouldn't reach here
  return "tool_handler";
}

#### Agent Graph

Finally, we can assemble all components:

In [ ]:
import { StateGraph, START, END } from "@langchain/langgraph";

// Build workflow
const overallWorkflow = new StateGraph<State>({
  channels: StateSchema
})
  .addNode("llm_call", llmCall)
  .addNode("tool_handler", toolHandler);

// Add edges
overallWorkflow.addEdge(START, "llm_call");
overallWorkflow.addConditionalEdges(
  "llm_call",
  shouldContinue,
  {
    "tool_handler": "tool_handler",
    [END]: END,
  }
);
overallWorkflow.addEdge("tool_handler", "llm_call");

// Compile the agent
const agent = overallWorkflow.compile();

// In a TypeScript environment, we would visualize it with:
// agent.showGraph();

// For the notebook, we'd output an image similar to the Python version


This creates a graph that:
1. Starts with an LLM decision
2. Conditionally routes to tool execution or termination
3. After tool execution, returns to LLM for the next decision
4. Repeats until completion or no tool is called


### Combine workflow with our agent

We can combine the router and the agent.

In [ ]:
// Combine the triage router and agent workflow
const combinedWorkflow = new StateGraph<State>({
  channels: StateSchema
})
  .addNode("triage_router", triageRouter)
  .addNode("response_agent", agent)
  .addEdge(START, "triage_router");

// Compile the combined workflow
const finalWorkflow = combinedWorkflow.compile();

// For the notebook, we'd output an image similar to the Python version
// In a TypeScript environment, we would visualize it with:
// finalWorkflow.showGraph();

This is a higher-level composition where:
1. First, the triage router analyzes the email
2. If needed, the response agent handles crafting a response
3. The workflow ends when either the triage decides no response is needed or the response agent completes

In [ ]:
// Test email input
const testEmailInput = {
  author: "System Admin <sysadmin@company.com>",
  to: "Development Team <dev@company.com>",
  subject: "Scheduled maintenance - database downtime",
  email_thread: "Hi team,\n\nThis is a reminder that we'll be performing scheduled maintenance on the production database tonight from 2AM to 4AM EST. During this time, all database services will be unavailable.\n\nPlease plan your work accordingly and ensure no critical deployments are scheduled during this window.\n\nThanks,\nSystem Admin Team"
};

// Run the agent
const response = await finalWorkflow.invoke({
  email_input: testEmailInput,
  messages: [],
  classification_decision: null  // Will be set during execution
});

// In a real TypeScript environment, we would display messages like:
// response.messages.forEach(m => console.log(m));


In [ ]:
// Second test email input
const testEmailInput2 = {
  author: "Alice Smith <alice.smith@company.com>",
  to: "John Doe <john.doe@company.com>",
  subject: "Quick question about API documentation",
  email_thread: "Hi John,\nI was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?\nSpecifically, I'm looking at:\n- /auth/refresh\n- /auth/validate\nThanks!\nAlice"
};

// Run the agent with the second test input
const response2 = await finalWorkflow.invoke({
  email_input: testEmailInput2,
  messages: [],
  classification_decision: null  // Will be set during execution
});

// In a real TypeScript environment, we would display messages like:
// response2.messages.forEach(m => console.log(m));


## Testing with Local Deployment

You can find the file for our agent in the `src` directory:

* `src/email_assistant.ts`

You can test them locally in LangGraph Studio by running:

```bash
pnpm agent
```

Example e-mail you can test:

In [ ]:
const emailInputExample = {
  author: "Alice Smith <alice.smith@company.com>",
  to: "John Doe <john.doe@company.com>",
  subject: "Quick question about API documentation",
  email_thread: "Hi John,\nI was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?\nSpecifically, I'm looking at:\n- /auth/refresh\n- /auth/validate\nThanks!\nAlice"
};

![studio-img](img/studio.png)